In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
import albumentations.pytorch
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from tqdm import tqdm

import wandb

config={
    "lr": 2e-4,
    "dropout": 0.5,
    "architecture": "efficientnet-b6 + 3way",
    "dataset": "vanilla + mask",
    "augmentation" : "None",
    "age_weightedloss" : [1.0, 1.2, 3.],
    "gender_weightedloss" : [1.4, 1.0],
    "gamma" : 0.85,
    "batch_size" : 64,
    "epochs" : 15,
    'model' : 'efficientnet-b6'
}

wandb.init(project="MaskClassification", name=config['model'], config=config)



wandb: Currently logged in as: youkind (use `wandb login --relogin` to force relogin)

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [2]:
print('CUDA GPU available : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')
device = torch.device("cuda")
CUDA_LAUNCH_BLOCKING=1

CUDA GPU available : True
1 GPU(s) is(are) allocated


In [3]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

labels_to_class = {}
it = [(m, g, a) for m in [0,1,2] for g in [0, 1] for a in [0, 1, 2]]
for i, (m, g, a) in enumerate(it):
    labels_to_class[(m, g, a)] = i

## Dataset

In [4]:
class TrainDataset(Dataset):
    def __init__(self, train_dir, is_Train=True, transform=None):
        super().__init__()
        
        csv_path = os.path.join(train_dir, 'train.csv')
        csv = pd.read_csv(csv_path)
        self.image_dir = os.path.join(train_dir, 'images')
        self.transform = transform
        self.image_path = []
        path = csv['path']
        
        for p in path:
            images = [os.path.join(*[self.image_dir, p, image]) for image in os.listdir(os.path.join(self.image_dir, p)) if not image[:1] == '.']
            for image in images:
                self.image_path.append(image)

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_name = self.image_path[idx]
        image = cv2.imread(image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        features = image_name.split('/')[-2:]
        
        masktoint = {'m' : 0, 'i' : 1, 'n' : 2}
        gendertoint = {'male' : 0, 'female' : 1}
        
        mask = masktoint[features[1][0]]
        age = int(features[0].split('_')[-1])
        gender = gendertoint[features[0].split('_')[1]]
        
        if age >= 58: # 원래 60
            age = 2
        elif age >= 30: # 원래 30
            age = 1
        else:
            age = 0        
        
        if self.transform:
            image = self.transform(image=image)['image']
        
        return image, (mask, gender, age)

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = cv2.imread(self.img_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [5]:
from torch.utils.data.dataset import random_split
tfms = A.Compose([
        A.augmentations.crops.transforms.CenterCrop(400, 360, p=1.0),
        A.augmentations.geometric.resize.Resize(224, 224, interpolation=1, p=1),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.6),
        A.RandomBrightnessContrast(brightness_limit=0.1, p=0.6),
        A.GaussNoise(p=0.5),
        A.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        A.pytorch.transforms.ToTensorV2(),
    ])
tfms_test = A.Compose([
        A.augmentations.crops.transforms.CenterCrop(400, 360, p=1.0),
        A.augmentations.geometric.resize.Resize(224, 224, interpolation=1, p=1),
        A.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        A.pytorch.transforms.ToTensorV2(),
    ])

person_idx = list(range(2700))
np.random.shuffle(person_idx)
n_val_person = int(2700 * 0.2)
val_person_idx = person_idx[:n_val_person]

train_indices = []
val_indices = []
for i in range(2700*15):
    if i // 15 not in val_person_idx:
        train_indices.append(i)
    else:
        val_indices.append(i)
        
dataset = TrainDataset(train_dir, transform=tfms)
# train_dataset, val_dataset = random_split(dataset, [int(len(dataset)*0.8),int(len(dataset)*0.2)])
# print(len(dataset))
# plt.imshow(np.array(train[1][0].permute(1,2,0)))
train_loader = DataLoader(dataset, batch_size=config['batch_size'], pin_memory=True, num_workers=4, sampler=train_indices)
val_loader   = DataLoader(dataset, num_workers=4, batch_size=config['batch_size'], pin_memory=True, sampler=val_indices)

In [6]:
# plt.imshow(np.array(train_dataset[312][0]['image'].permute(1,2,0)))

## Model

In [7]:
from efficientnet_pytorch import EfficientNet

class Way3(nn.Module):
    def __init__(self):
        super().__init__()
#         self.convnet = timm.create_model(config['model'], pretrained=True, num_classes=512)
        self.convnet = EfficientNet.from_pretrained(config['model'], num_classes=512).to(device)
        self.mask = nn.Sequential(
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(512, 128),
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(128, 3),
            nn.Softmax(dim=1)
        )
        self.gender = nn.Sequential(
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(512, 128),
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )
        self.age = nn.Sequential(
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(512, 128),
            nn.LeakyReLU(),
            nn.Dropout(p=config['dropout'], inplace=False),
            nn.Linear(128, 3),
            nn.Softmax(dim=1)
        )
        self.best_f1 = 0
        
    def forward(self, x):
        features = self.convnet(x)
        mask = self.mask(features)
        gender = self.gender(features)
        age = self.age(features)
        
        return mask, gender, age
    
model = Way3().to(device)

Loaded pretrained weights for efficientnet-b6


In [8]:
import torch.optim as optim 

gender_weight = torch.tensor(config["gender_weightedloss"])
age_weight = torch.tensor(config["age_weightedloss"])
                             
criterion = nn.CrossEntropyLoss()
gender_criterion = nn.CrossEntropyLoss(weight=gender_weight).to(device)
age_criterion = nn.CrossEntropyLoss(weight=age_weight).to(device)
# b_criterion = nn.BCEWithLogitsLoss()
# criterion = FocalLoss()
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=config['gamma'], verbose=True)

Adjusting learning rate of group 0 to 2.0000e-04.


In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def check(loader, length, model, device):
    model.eval()
    val_loss = 0.0
    counter = 0
    y_true = []
    y_predicted = []

    m_acc = []
    g_acc = []
    a_acc = []
    with torch.no_grad():
        for (inputs, (m, g, a)) in val_loader:
            counter += 1
            
            for mask, gender, age in zip(m, g, a):
                answer = labels_to_class[(mask.item(), gender.item(), age.item())]
                y_true.append(answer)
            
            inputs = inputs.to(device=device)
            m = m.to(device)
            g = g.to(device)
            a = a.to(device)

            m_pred, g_pred, a_pred = model(inputs)
    
            m_loss = criterion(m_pred, m)
            g_loss = gender_criterion(g_pred, g)
            a_loss = age_criterion(a_pred, a) # data imbalance
            
            loss = (g_loss+a_loss+m_loss)
            
            val_loss += loss.item()
            
            m_argmax = m_pred.detach().cpu().numpy().argmax(1)
            g_argmax = g_pred.detach().cpu().numpy().argmax(1)
            a_argmax = a_pred.detach().cpu().numpy().argmax(1)

            m_acc.append(accuracy_score(m_argmax, m.detach().cpu().numpy()))
            g_acc.append(accuracy_score(g_argmax, g.detach().cpu().numpy()))
            a_acc.append(accuracy_score(a_argmax, a.detach().cpu().numpy()))

            for mask, gender, age in zip(m_argmax, g_argmax, a_argmax):
                predicted = labels_to_class[(mask.item(), gender.item(), age.item())]
                y_predicted.append(predicted)
    
    
    cm = confusion_matrix(y_true, y_predicted)
    F1 = []
    for c in range(18):
        precision = cm[c][c] / np.sum(cm, axis=0)[c]
        recall = cm[c][c] / np.sum(cm, axis=1)[c]
        F1.append(2 * precision * recall / (precision + recall))
    macro_F1 = np.mean(F1)

    s = 0
    for c in range(18):
        s += cm[c][c]
        
    print("< VALIDATION >")
    print("*"*73)
    print("Validation Loss :", val_loss/counter)
    print("-"*73)
    print("Total Accuracy")
    print(s / length * 100, "%")
    print("-"*73)
    print("Class Accuracy")
    print("Mask   :", np.mean(m_acc)*100, "%")
    print("Gender :", np.mean(g_acc)*100, "%")
    print("Age    :", np.mean(a_acc)*100, "%")
    print("-"*73)
    print("Confusion Matrix")
    for row in cm:
        print(row)
    print("-"*73)
    print("Validation F1 score :" , macro_F1)
    if model.best_f1 < macro_F1:
        model.best_f1 = macro_F1
        torch.save(model.state_dict(), '/opt/ml/weights/3way/{}/{:.4f}.pt'.format(config['model'], model.best_f1))
        print("model saved!")
    print("*"*73)
    print()
    wandb.log({
        "Validation Loss" : val_loss/counter, 
        "Validation Total Accuracy" : s / length *100, 
        "Validation F1" : macro_F1,
        "Mask Accuracy" : np.mean(m_acc)*100,
        "Gender Accuracy" : np.mean(g_acc)*100,
        "Age Accuracy" : np.mean(a_acc)*100,
    })
        
    model.train()

In [10]:

folder = '/opt/ml/weights/3way/{}'.format(config['model'])
if not os.path.exists(folder):
    os.mkdir(folder)

for epoch in range(config['epochs']):
    print("Epoch :", epoch + 1)
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    total = 0
    total_it = int(len(train_indices)/train_loader.batch_size)
    prog_bar = tqdm(enumerate(train_loader), total=total_it)
    for i, (inputs, (m, g, a)) in prog_bar:
        
        counter += 1
        optimizer.zero_grad()
        inputs = inputs.to(device)
        m_pred, g_pred, a_pred = model(inputs)
        
        
        m = m.to(device)
        g = g.to(device)
        a = a.to(device)
        total += m.size(0)
        
        
        m_loss = criterion(m_pred, m)
        g_loss = gender_criterion(g_pred, g)
        a_loss = age_criterion(a_pred, a) # data imbalance
        
        
        loss = (g_loss+a_loss+m_loss)
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()
        
        if i == total_it//2 or i == total_it-1:
            train_loss = train_running_loss / counter

            print("Loss :", train_loss)
            wandb.log({"Train Loss" : train_loss})
            
            check(val_loader, len(val_indices), model, device)
    scheduler.step()
    train_running_loss = 0.0
    train_running_correct = 0
print("Finish")

Epoch : 1


 50%|█████     | 253/506 [03:08<03:06,  1.36it/s]

Loss : 2.210517991246201


<ipython-input-9-a00a7d22706f>:53: RuntimeWarning: invalid value encountered in long_scalars
  precision = cm[c][c] / np.sum(cm, axis=0)[c]
 50%|█████     | 254/506 [03:33<33:36,  8.00s/it]

< VALIDATION >
*************************************************************************
Validation Loss : 2.4499353189168014
-------------------------------------------------------------------------
Total Accuracy
23.703703703703706 %
-------------------------------------------------------------------------
Class Accuracy
Mask   : 97.98501749781278 %
Gender : 60.71604330708661 %
Age    : 44.82037401574803 %
-------------------------------------------------------------------------
Confusion Matrix
[  0   0   0 556   0   0   0   0   0   4   0   0   0   0   0   0   0   0]
[  0   0   0 319   0   0   0   0   0  11   0   0   0   0   0   0   0   0]
[  0   0   0 161   0   0   0   0   0   4   0   0   0   0   0   0   0   0]
[  0   0   0 639   0   0   0   0   0   6   0   0   0   0   0   0   0   0]
[  0   0   0 712   0   0   0   0   0  18   0   0   0   0   0   0   0   0]
[  0   0   0 259   0   0   0   0   0  11   0   0   0   0   0   0   0   0]
[  0   0   0   8   0   0   0   0   0 547   0   0   0 

 74%|███████▍  | 376/506 [05:03<01:44,  1.24it/s]


KeyboardInterrupt: 

## Testing

In [ ]:
model.load_state_dict(torch.load('/opt/ml/weights/3way/{}/{:.4f}.pt'.format(config['model'], model.best_f1)))
# model.load_state_dict(torch.load('/opt/ml/weights/3way/{}/0.9841404744270768.pt'.format(config['model']))

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, tfms_test)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=64,
    num_workers=4
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# device = torch.device('cuda')

model.eval()
device = torch.device("cuda:0")
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

prog_bar = tqdm(enumerate(loader), total=int(len(dataset)/loader.batch_size))
for i, images in prog_bar:
    with torch.no_grad():
        images = images.to(device)
        m_pred, g_pred, a_pred = model(images)
#         print(images.shape)
#         print(m_pred.shape)
        m_argmax = m_pred.detach().cpu().numpy().argmax(1)
        g_argmax = g_pred.detach().cpu().numpy().argmax(1)
        a_argmax = a_pred.detach().cpu().numpy().argmax(1)
        
        for mask, gender, age in zip(m_argmax, g_argmax, a_argmax):
            predicted = labels_to_class[(mask.item(), gender.item(), age.item())]
            all_predictions.append(predicted)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
idx = 1020
image = cv2.imread(image_paths[idx])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

image = tfms_test(image=image)['image'].to(device)
image = image.unsqueeze(0)
m_pred, g_pred, a_pred = model(image)

m_argmax = m_pred.detach().cpu().numpy().argmax(1)
g_argmax = g_pred.detach().cpu().numpy().argmax(1)
a_argmax = a_pred.detach().cpu().numpy().argmax(1)

#
masklabel = {0: "Mask", 1: "Incorrect", 2: "Normal"}
genderlabel = {0: "Male", 1: "Female"}
agelabel = {0: "~ 30", 1: "30 ~ 60", 2: "60 ~"}
#
print(masklabel[m_argmax[0]]+ " :", (m_pred[0][m_argmax].item())*100, '%')
print(genderlabel[g_argmax[0]]+ " :", (g_pred[0][g_argmax].item())*100, '%')
print(agelabel[a_argmax[0]]+ " :", (a_pred[0][a_argmax].item())*100, '%')
